In [1]:
import sys
from pyspark import SparkContext
import time

# Finds out the index of "name" in the array firstLine
# returns -1 if it cannot find it
def findCol(firstLine, name):
    if name in firstLine:
        return firstLine.index(name)
    else:
        return -1


#### Driver program

# start spark with 1 worker thread
sc = SparkContext("local[1]")
sc.setLogLevel("ERROR")


# read the input file into an RDD[String]
# find the data description in the link 
# http://webs.ucm.es/info/cliwoc/content/CLIWOC15all.htm
wholeFile = sc.textFile("./data/CLIWOC15.csv")

# The first line of the file defines the name of each column in the cvs file
# We store it as an array in the driver program
firstLine = (
    wholeFile.filter(lambda x: "RecID" in x).collect()[0].replace('"', "").split(",")
)

# filter out the first line from the initial RDD
entries = wholeFile.filter(lambda x: not ("RecID" in x))

# split each line into an array of items
entries = entries.map(lambda x: x.split(","))

# keep the RDD in memory
entries.cache()

##### Create an RDD that contains all nationalities observed in the
##### different entries

# Information about the nationality is provided in the column named
# "Nationality"

# First find the index of the column corresponding to the "Nationality"
column_index = findCol(firstLine, "Nationality")
print("{} corresponds to column {}".format("Nationality", column_index))

# Use 'map' to create a RDD with all nationalities and 'distinct' to remove duplicates
nationalities = entries.map(lambda x: x[column_index])
nationalities = nationalities.map(lambda x: x.replace(" ", "")).distinct()
# Display the 5 first nationalities
print("A few examples of nationalities:")
for elem in nationalities.sortBy(lambda x: x).take(5):
    print(elem)
    

Nationality corresponds to column 20
A few examples of nationalities:
"American"
"British"
"Danish"
"Dutch"
"French"


In [4]:
# counting the total number of observations included in the dataset (each line corresponds to one observation)

from timeit import default_timer as timer

start = timer()
count = entries.count()
end = timer()
print("Total number of observations is: ", count)

print("Time: ", end - start)


Total number of observations is:  280280
Time:  3.572205099999337


In [5]:
# counting the number of years over which observations have been made (Column "Year" should be used)

start = timer()
column_index_year = findCol(firstLine, "Year")
year_observations = entries.map(lambda x: x[column_index_year])
year_observations = year_observations.filter(lambda x: x != "NA").distinct()
count = year_observations.count()
end = timer()
print("Total number: ", count)
print("Time: ", end - start)


Total number:  118
Time:  4.852775900000779


In [6]:
# displaying the oldest and the newest year of observation

start = timer()
print("The newest year is:", year_observations.max())
print("The oldest year is:", year_observations.min())
end = timer()
print("Time: ", end - start, "seg")

The newest year is: 1855
The oldest year is: 1662
Time:  0.8247226000003138 seg


In [7]:
# displaying the years with the minimum and the maximum number of observations (and the corresponding number of observations)

import operator
from operator import add

start = timer()
# mapping all the rows with duplicates for the column of Year
year_observations = entries.map(lambda x: x[column_index_year])  
year_observations = year_observations.map(lambda x: (x, 1))
year_observations_tuples = year_observations.foldByKey(0, add).collect()
dictionary = dict(year_observations_tuples)

year_max_observations = max(dictionary.items(), key=operator.itemgetter(1))
print("The year with the maximum number of observations is", year_max_observations)

year_min_observations = min(dictionary.items(), key=operator.itemgetter(1))
print("The year with the minimum number of observations is", year_min_observations)
end = timer()

print("Time: ", end - start)


The year with the maximum number of observations is ('1778', 8509)
The year with the minimum number of observations is ('1747', 4)
Time:  4.427495199999612


In [9]:
# counting distinct departure places (column "VoyageFrom") using two methods (i.e., using the function distinct() 
# or reduceByKey()) and compare the execution time.

# 1. with distinct()

start = timer()
column_index_voyageFrom = findCol(firstLine, "VoyageFrom")
voyageFrom_observations = entries.map(lambda x: x[column_index_voyageFrom])
voyageFrom_observations = voyageFrom_observations.filter(lambda x: x != "NA").distinct()
count = voyageFrom_observations.count()
end = timer()
print("The number of distinct departure places is", count)
print("Time: ", end - start)



The number of distinct departure places is 995
Time:  4.620365299999321


In [11]:
# with reduceByKey()

start = timer()
column_index_voyageFrom = findCol(firstLine, "VoyageFrom")
voyageFrom_observations = entries.map(lambda x: x[column_index_voyageFrom])
voyageFrom_observations = voyageFrom_observations.filter(lambda x: x != "NA")
voyageFrom_observations = voyageFrom_observations.map(lambda x: (x, 1))
voyageFrom_observations = voyageFrom_observations.reduceByKey(lambda accum, n: accum + n)
count = voyageFrom_observations.count()
end = timer()

print("The number of distinct departure places is", count)
print("Time: ", end - start)


The number of distinct departure places is 995
Time:  4.649124700001266


In [13]:
# displaying the 10 most popular departure places

start = timer()
column_index_voyageFrom = findCol(firstLine, "VoyageFrom")
voyageFrom_observations = entries.map(lambda x: x[column_index_voyageFrom])
voyageFrom_observations = voyageFrom_observations.filter(lambda x: x != "NA")
voyageFrom_observations = voyageFrom_observations.map(lambda x: (x, 1))
voyageFrom_observations = voyageFrom_observations.reduceByKey(lambda accum, n: accum + n)

# Display the 10 most famous departures
print("The Top-10 departure places are:")
for elem in voyageFrom_observations.sortBy(lambda x: x[1], ascending=False).take(10):
    print(elem)

end = timer()
print("Time: ", end - start)


The Top-10 departure places are:
('"Batavia"', 25920)
('"Rotterdam"', 9757)
('"Nederland"', 8697)
('"Montevideo"', 8680)
('"La Coruña"', 8463)
('"Spithead"', 8298)
('"LA HABANA"', 7906)
('"LA CORUÑA"', 7657)
('"CÁDIZ"', 7522)
('"Nieuwediep"', 6713)
Time:  5.533315299999231


In [14]:
# displaying the 10 roads (defined by a pair "VoyageFrom" and "VoyageTo") the most often taken

start = timer()
column_index_voyageFrom = findCol(firstLine, "VoyageFrom")
column_index_voyageTo = findCol(firstLine, "VoyageTo")
voyageFrom_voyageTo = entries.map(
    lambda x: ((x[column_index_voyageFrom], x[column_index_voyageTo]), 1)
)
voyageFrom_voyageTo = voyageFrom_voyageTo.filter(lambda x: "NA" not in x[0])
voyageFrom_voyageTo = voyageFrom_voyageTo.reduceByKey(lambda x, y: x + y)
for elem in voyageFrom_voyageTo.sortBy(lambda x: -x[1]).take(10):
    print(elem)

end = timer()
print("Time: ", end - start)


(('"La Coruña"', '"Montevideo"'), 8463)
(('"Montevideo"', '"La Coruña"'), 8382)
(('"LA CORUÑA"', '"LA HABANA"'), 7525)
(('"Rotterdam"', '"Batavia"'), 7341)
(('"LA HABANA"', '"LA CORUÑA"'), 6068)
(('"Batavia"', '"Rotterdam"'), 5256)
(('"Nieuwediep"', '"Batavia"'), 5256)
(('"Batavia"', '"Nieuwediep"'), 4564)
(('"Nederland"', '"Batavia"'), 3996)
(('"Batavia"', '"Nederland"'), 3534)
Time:  5.524020799999562
